In [17]:
import whisper_s2t
import os
print(whisper_s2t.__file__)

/Users/nafi/miniconda3/envs/stt_env/lib/python3.10/site-packages/whisper_s2t/__init__.py


In [18]:
import datetime
import os
import time  # Add this import

def record_audio(duration=10, sample_rate=16000):
    # Initialize PyAudio
    p = pyaudio.PyAudio()
    
    # List available input devices
    print("\nAvailable Input Devices:")
    for i in range(p.get_device_count()):
        dev = p.get_device_info_by_index(i)
        if dev.get('maxInputChannels') > 0:  # Only show input devices
            print(f"Device {i}: {dev.get('name')}")
    
    # Open stream with better settings
    stream = p.open(format=pyaudio.paInt16,
                   channels=1,
                   rate=sample_rate,
                   input=True,
                   frames_per_buffer=1024,
                   input_device_index=None)  # Uses default device
    
    print("\nPreparing to record...")
    print("3...")
    time.sleep(1)
    print("2...")
    time.sleep(1)
    print("1...")
    time.sleep(1)
    print(f"Recording for {duration} seconds...")
    
    frames = []
    
    # Add audio level feedback while recording
    for _ in range(0, int(sample_rate / 1024 * duration)):
        data = stream.read(1024, exception_on_overflow=False)
        frames.append(data)
        # Show audio level (optional)
        audio_data = np.frombuffer(data, dtype=np.int16)
        level = np.abs(audio_data).mean()
        bars = int(50 * level / 10000)
        print(f"\rAudio Level: {'|' * bars}{' ' * (50-bars)}", end='', flush=True)
    
    print("\nRecording finished!")
    
    # Stop and close the stream
    stream.stop_stream()
    stream.close()
    p.terminate()
    
    # Generate filename with timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"recording_{timestamp}.wav"
    
    # Save to current directory
    wf = wave.open(filename, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(2)  # for paInt16
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    print(f"\nAudio saved as: {filename}")
    return filename

# Load the model
print("Loading Whisper model...")
model = whisper_s2t.load_model(
    model_identifier="large-v2", 
    backend='CTranslate2',
    device="cpu",
    compute_type="int8"
)

while True:
    # Ask user if they want to record
    response = input("\nPress Enter to start recording (or 'q' to quit): ")
    if response.lower() == 'q':
        break
        
    # Record audio
    audio_file = record_audio(duration=10)  # You can adjust duration here
    
    print("\nTranscribing...")
    out = model.transcribe_with_vad([audio_file],
                                   lang_codes=['en'],
                                   tasks=['transcribe'],
                                   initial_prompts=[None],
                                   batch_size=32)
    
    print("\nTranscription:", out[0][0]['text'])

Loading Whisper model...


Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 83468.74it/s]



Available Input Devices:
Device 0: MacBook Pro Microphone
Device 2: Microsoft Teams Audio

Preparing to record...
3...
2...
1...
Recording for 10 seconds...
Audio Level:                                                   
Recording finished!

Audio saved as: recording_20250101_211115.wav

Transcribing...


Transcribing: 100%|██████████| 100/100 [00:04<00:00, 20.71it/s]



Transcription: I have had rice at dinner. What did you have?
